In [30]:
import pandas as pd
import numpy as np

biden = pd.read_csv("hashtag_joebiden.csv" , lineterminator='\n')
trump = pd.read_csv("hashtag_donaldtrump.csv", lineterminator='\n')

print(biden.head())
print(trump.head())
print(biden.columns)
trump.info()
biden.info()

            created_at      tweet_id  \
0  2020-10-15 00:00:01  1.316529e+18   
1  2020-10-15 00:00:18  1.316529e+18   
2  2020-10-15 00:00:20  1.316529e+18   
3  2020-10-15 00:00:21  1.316529e+18   
4  2020-10-15 00:00:22  1.316529e+18   

                                               tweet  likes  retweet_count  \
0  #Elecciones2020 | En #Florida: #JoeBiden dice ...    0.0            0.0   
1  #HunterBiden #HunterBidenEmails #JoeBiden #Joe...    0.0            0.0   
2  @IslandGirlPRV @BradBeauregardJ @MeidasTouch T...    0.0            0.0   
3  @chrislongview Watching and setting dvr. Let’s...    0.0            0.0   
4  #censorship #HunterBiden #Biden #BidenEmails #...    1.0            0.0   

               source       user_id           user_name user_screen_name  \
0           TweetDeck  3.606665e+08  El Sol Latino News  elsollatinonews   
1    Twitter for iPad  8.099044e+08         Cheri A. 🇺🇸     Biloximeemaw   
2     Twitter Web App  3.494182e+09          Flag Waver      F

In [31]:
#Remove unneeded columns
trump = trump.drop(columns=['user_description','user_join_date'])
biden = biden.drop(columns=['user_description','user_join_date'])

#Rename column names
trump = trump.rename(columns={"likes": "Likes", "retweet_count": "Retweets",
                              "state": "State", "user_followers_count": "Followers"})
biden = biden.rename(columns={"likes": "Likes", "retweet_count": "Retweets",
                                "state": "State", "user_followers_count": "Followers"})


trump['candidate'] = 'Trump'
biden['candidate'] = 'Biden'
#concat the two data sets
data = pd.concat([trump, biden])
print('Data shape:', data.shape)
print('First three rows:')
print(data.tail(3))
data.info()

Data shape: (1747805, 20)
First three rows:
                 created_at      tweet_id  \
776883  2020-11-08 23:59:41  1.325589e+18   
776884  2020-11-08 23:59:52  1.325589e+18   
776885  2020-11-08 23:59:58  1.325589e+18   

                                                    tweet  Likes  Retweets  \
776883  L'OTAN va sortir de sa léthargie et redevenir ...   48.0      14.0   
776884  🌎\n\n“#congiuntifuoriregione”\n\n‘Sono felice ...    1.0       1.0   
776885  Ik moet zeggen dat ik #Biden "the lesser of tw...    0.0       0.0   

                     source       user_id  \
776883  Twitter for Android  7.819183e+17   
776884   Twitter for iPhone  5.293315e+08   
776885  Twitter for Android  5.863863e+08   

                                         user_name user_screen_name  \
776883  🇫🇷 Alt-Droite (matricule 6921) ✝️ 🇬🇷 🇮🇹 🇦🇲    CtrlAltDroite   
776884                                 Angelo Tani       AngeloTani   
776885                                         Job          _JobO__ 

In [32]:
print(data['tweet'])

#Calculate how many rows would be dropped if apply dropna to the entire data set
original_row_count = data.shape[0]
remaining_row_count = data.dropna().shape[0]
rows_eliminated = original_row_count - remaining_row_count
print(f"Number of rows that would be eliminated: {rows_eliminated}")


# data.dropna(inplace=True)
# data['country'].value_counts()

# Fill missing values in numeric columns with 0 for non-important columns
numeric_columns = ['Likes', 'Retweets', 'Followers']
data[numeric_columns] = data[numeric_columns].fillna(0)

# Fill missing text columns with an empty string for non-important columns
text_columns = ['source', 'user_name', 'user_screen_name', 'country']
data[text_columns] = data[text_columns].fillna('')

data = data[data['tweet'].notna()]

import re

def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#','', text)  # Remove mentions and hashtags
    text = re.sub(r"[^a-zA-Z0-9\s]", '', text)  # Remove special characters
    return text.lower().strip()  # Convert to lowercase and strip extra spaces

data['tweet'] = data['tweet'].apply(clean_text)

data = data.drop_duplicates(subset=['tweet_id'])
data = data.reset_index(drop=True)
data.info()

0         #Elecciones2020 | En #Florida: #JoeBiden dice ...
1         Usa 2020, Trump contro Facebook e Twitter: cop...
2         #Trump: As a student I used to hear for years,...
3         2 hours since last tweet from #Trump! Maybe he...
4         You get a tie! And you get a tie! #Trump ‘s ra...
                                ...                        
776881    Stop laying @CNN !\n#Paris and #London dont gi...
776882    Ωχ ελπίζω να μη μας βγει  σαν τους οπαδούς του...
776883    L'OTAN va sortir de sa léthargie et redevenir ...
776884    🌎\n\n“#congiuntifuoriregione”\n\n‘Sono felice ...
776885    Ik moet zeggen dat ik #Biden "the lesser of tw...
Name: tweet, Length: 1747805, dtype: object
Number of rows that would be eliminated: 1383967
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522909 entries, 0 to 1522908
Data columns (total 20 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   created_at        1522909 non-n